# 理解基于区域的物体检测模型
翻译自：https://medium.com/@jonathan_hui/what-do-we-learn-from-region-based-object-detectors-faster-r-cnn-r-fcn-fpn-7e354377a7c9

在这个系列中，我们将尝试深入理解物体检测任务。第一部分我们将介绍基于区域的物体检测模型，包括Fast R-CNN，Faster R-CNN，R-FCN以及FPN。第二部分我们将学习单次检测模型。第三部分我们将讨论性能以及一些实现上的问题。通过学习本文，我们将学习内部工作原理，注意事项以及改善点。通过这些能够更加深入地理解物体检测任务。<br>

- 第一部分：理解基于区域的物体检测模型工作原理（Faster R-CNN, R-FCN, FPN）
- 第二部分：理解单次检测模型工作原理（SSD,YOLO）,FPN以及Focal loss
- 第三部分：设计建议和经验，物体检测的发展方向

### 滑动窗口检测器
自从AlexNet赢得了2012年ILSVRC挑战赛的冠军，CNN在图像分类领域处于主宰地位。物体检测任务的一个最暴力的方法就是从左到右、从上到下的滑动窗口，在每个窗口做分类任务。为了在不同的视角和距离检测物体种类，会使用不同尺寸和纵横比的窗口。<br>
![avater](https://cdn-images-1.medium.com/max/1000/1*-GaZ8hGBKsbtGfRJqvOVHQ.jpeg)

我们基于滑动窗口来提取图片的局部块。这些局部有可能是扭曲的，因为很多分类器只能处理固定尺寸的图像。然而，这并不会影响分类准确率，因为分类器在训练时就会将扭曲的图像纳入训练集。
![avater](https://cdn-images-1.medium.com/max/800/1*A7DE4HKukbXpQqwvCaLOEQ.jpeg)
扭曲的图像局部块输入CNN分类器提取4096维的特征向量，然后应用SVM分类器用于分类，应用另一个回归模型用于回归边界框。
![avater](https://cdn-images-1.medium.com/max/800/1*BYSA3iip3Cdr0L_x5r468A.png)
下面是伪代码。我们创建了很多窗口来检测不同位置的不同物体。为了提高性能，一个很直接的方法是减少窗口数量。
```python
for window in windows:
    patch = get_patch(image,window)
    results = detector(patch)
```

### 选择查找（Selective Search）
不同于暴力方法，我们可以使用一个区域目标方法，即通过创建**感兴趣区域（ROIs）**来做目标检测。在**选择查找（SS）**中，我们开始令每一个像素都是一个组。然后，我们计算每个组的质地，然后将彼此最近的合并为一个组。为了避免一个区域吞噬掉另一个区域，我们优先合并较小的组。我们持续合并组直到所有的组都合并完毕。在下图第一行，我们展示了区域是如何合并的，第二行展示了合并过程中得到的所有可能的ROIs。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*_8BNWWwyod1LWUdzcAUr8w.png)

### R-CNN 
R-CNN通过使用区域提取方法得到了2000个ROIs。这些区域之后扭曲变形为同样尺寸的图像单独输入到CNN网络中。网络的最后是全连接层，用于分类物体和回归边界框。<br>
![avater](https://cdn-images-1.medium.com/max/1000/1*Wmw21tBUez37bj-1ws7XEw.jpeg)
下面是系统的流程。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*ciyhZpgEvxDm1YxZd1SJWg.png)
通过更少以及质量更高的ROIs,R-CNN相对于滑动窗口，更加快速和准确。<br>
```python
ROIs = region_proposal(image)
for ROI in ROIs:
    patch = get_patch(image, ROI)
    results = detector(patch)
```

### 边界框回归器
区域提取模型的计算复杂度还是很高。为了提高速度，我们通常采用一种更加简单的区域提取方法来创建ROIs，之后输入线性回归器用于回归边界框。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*rvPyjhiVQnOm3yOqSDUKuA.jpeg)

###  Fast R-CNN
R-CNN需要很多精确的目标框而且很多区域彼此有重叠。R-CNN在训练和预测时速度都很慢。如果我们有2000个目标框，每个都单独输入CNN做处理，也就是说我们将特征提取过程重复了2000遍。<br>
为了解决这个问题，Fast R-CNN的做法是不在每个目标框上做特征提取，而是首先对整张图像做特征提取。然后在特征图上结合此前区域提取方法如SS创建的ROIs得到特征图目标框，之后我们使用**ROI pooling**将目标框处理成固定的尺寸，然后利用全连接层做分类和定位任务。由于只做了一次特征提取，Fast R-CNN大大地提高了速度。<br>
![avater](https://cdn-images-1.medium.com/max/1000/1*Dd3-sugNKInTIv12u8cWkw.jpeg)
流程图如下：
![avater](https://cdn-images-1.medium.com/max/800/1*fLMNHfe_QFxW569s4eR7Dg.jpeg)
伪代码如下，没有了重复特征提取，速度得到大大提升。Fast R-CNN在训练时比R-CNN快10倍，在预测时比R-CNN快150倍。<br>
```python
feature_maps = proprocess(image)
ROIS = region_proposal(image)
for ROI in ROIs:
    patch = roi_pooling(feature_maps, ROI)
    results = detector2(patch)
```
关于Fast R-CNN最主要的一个点是整个网络可以通过设置**多任务损失**（分类损失和定位损失）进行端到端的训练。这可以提高准确率。

### ROI Pooling
因为Fast R-CNN使用了全连接层，我们应用ROI Pooing将不同尺寸的ROIs处理为预先定义好的同一尺寸。<br>
我们以一个简单的例子来说明，特征图尺寸是8x8，预定义尺寸是2x2。<br>

- 下图左上：我们得到的特征图
- 下图右上：将目标框（蓝色）和特征图重叠
- 下图左下：我们将目标框分割为目标维度。例如，这里我们的目标维度是2x2，因此我们将目标框分割成了四个区域，这四个区域有相似或相等的尺寸。
- 下图右下：寻找每个区域的最大值作为该区域的值。
![avater](https://cdn-images-1.medium.com/max/800/1*LLP4tKGsYGgAx3uPfmGdsw.png)
最终我们在一个ROI上得到了一个2x2的特征块，可以后接全连接层进行分类和回归任务。

### Faster R-CNN
Fast R-CNN需要一个附加的区域提取方法如SS。然而，算法在CPU上的运算速度非常慢。在测试中，Fast R-CNN需要使用2.3秒作出一个预测，其中2秒用于生成2000个ROIs。<br>
```python
feature_maps = proprocess(image)
ROIS = region_proposal(image)    #计算消耗很大
for ROI in ROIs:
    patch = roi_pooling(feature_maps, ROI)
    results = detector2(patch)
```
Faster R-CNN跟Fast R-CNN基本一致，除了将区域提取方法替换为一个中间深度网络，然后从特征图里提取ROIs。这个新的区域提取网络（RPN）更加高效，生成每张图像的ROIs只需要10ms。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*F-WbcUMpWSE1tdKRgew2Ug.png)
网络流程图是类似的，但是区域提取从普通方法替换为卷积网络（RPN）。<br>
![avater](https://cdn-images-1.medium.com/max/1000/1*0cxB2pAxQ0A7AhTl-YT2JQ.jpeg)

### 区域提取网络（RPN）
RPN接收第一个卷积网络的特征图作为输入。在这个特征图上应用3x3卷积核进行无类别的区域提取，如下面所示的ZF网络。其他的网络如VGG或者ResNet可以用于更加细致的特征提取，但是需要以速度为代价。ZF网络输出256个值，输入到两个全连接层中分别预测边界框和两个对象性得分。这个**对象性**用来衡量目标框是否包含物体。我们可以使用一个回归器来计算一个对象性得分，但是为了简单起见，Faster R-CNN使用一个分类器得到了两个可能的类别：一个是“有物体”，另一个是是没有物体（即背景）。<br>
![avater](https://cdn-images-1.medium.com/max/1000/1*z0OHn89t0bOIHwoIOwNDtg.jpeg)
对于特征图中的每一个位置，RPN会做k个假设。因此RPN在每个位置输出4xk个坐标以及2xk个得分。下图展示了在8x8特征图上应用3x3卷积得到8x8x3个ROIs（其中k=3）。下图右边展示的是在单个位置上的3个目标框。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*smu6PiCx4LaPwGIo3HG0GQ.jpeg)
这里，我们做出了3个假设，并在之后对假设坐修正。由于我们只需要找到一个正确的，没必要在初始化时做不同尺寸和形状的假设。因此，Faster R-CNN就没有随机假设目标框。相反，她预测相对于一些参考边界框（叫anchors）的偏移量$\delta{x}$, $\delta{y}$。我们对这些便宜值作出限制，因此我们的假设仍然集成了anchors。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*yF_FrZAkXA3XKFA-sf7XZw.png)
为了在每个位置上作出k个预测，我们在每个位置上需要k个anchors。每个预测结果都和一个特定的anchor相关，但是不同位置上的anchor尺寸是一样的。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*RJoauxGwUTF17ZANQmL8jw.png)
这些anchors需要精挑细选以便保持多样性，并且在不同的尺存和纵横比上覆盖真实目标。这可以指导初始训练得到更好的猜测结果并且允许每个预测得到一个特定形状的目标框。这使得早期训练更加稳定和快速。<br>
Faster R-CNN使用了更多的anchors。它使用了9个anchor框：在3种纵横比以及三种尺度的组合。在每个位置上应用9个anchor，会得到2x9个对象性得分，以及4x9个坐标点。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*PszFnq3rqa_CAhBrI94Eeg.png)
Anchors也可以叫作先验或者默认边界框。

### R-CNN模型表现
如下图所示，Faster R-CNN表现的最好。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*fO2MSeQxIVVUUp6csJ8oWg.jpeg)

### 基于区域的全卷积神经网络（R-FCN）
假定我们只有一个检测右眼的特征图。我们能用用它来定位一张脸吗？可以的。由于右眼会出现在人脸图的左上角，我们可以据此来定位人脸。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*gqxSBKVla8dzwADKgADpWg.jpeg)
如果我们人脸其他部位（如左眼，鼻子，嘴巴）的特征图，我们可以结合这些特征图获得人脸的更好定位。<br>
所以我们的困惑在哪儿？在Faster R-CNN中，检测器需要应用多个全连接网络来作出预测。如果有2000个ROIs,会带来很大的计算消耗。<br>
```python
feature_maps = process(image)
ROIs = region_proposal(feature_maps)
for ROI in ROIs:
    patch = roi_pooling(feature_maps,ROI)
    class_scores, box = detector(patch)   # Expensive!
    class_probabilities = softmax(class_scores)
```
R-FCN通过减少每个ROI上的工作量来提高速度。基于区域的特征图是独立于ROI的，可以单独计算。剩下的工作就很简单了，因此R-FCN比Faster R-CNN更快。<br>
```python
feature_maps = process(image)
ROIs = region_proposal(feature_maps)
score_maps = compute_score_map(feature_maps)
for ROI in ROIs:
    V = region_roi_pool(scores_maps, ROI)
    class_scores, box = average(V)   # Much simpler!
    class_probabilities = softmax(class_scores)
```
考虑一个5x5的特征图M，其中有一个蓝色方形物体。我们将方形物体均分为3x3的区域。现在，我们创建了一个新的特征图只检测方形的左上角（TL）。新的特征图示意如下图最右。只有黄色的单元格是激活的。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*S0enLblW1t7VK19E1Fs4lw.png)
由于我们将这个方形分成了9个部分，我们可以创建9个特征图分别用来检测对应的区域。这些特征图叫做位置敏感得分图，因为每个特征图只检测（输出得分）物体的一个局部。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*HaOHsDYAf8LU2YQ7D3ymOg.png)
我们定义下图中红色长方形区域是最终得到的ROI。我们将其划分成3x3的区域并且计算每个区域包含物体对应部分的概率。例如，左上ROI区域包含左眼的可能性。我们将结果存储到一个3x3的投票数组中。例如，vote_array[0][0]包含的得分表示我们是否找到了物体的左上部分。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*Ym6b1qS0pXpeRVMysvvukg.jpeg)
这个将得分图和ROI映射到投票数组的过程叫做**position sensitive ROI-pool**。这个过程跟ROI pool极像。我们不做进一步的讨论，不过你可以利用扩展阅读获得更多的信息。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*K4brSqensF8wL5i6JV1Eig.png)
在计算了所有位置的ROI pool之后，通过取平均得到类别的得分。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*ZJiWcIl2DUyx1-ZqArw33A.png)
假定我们有C类需要检测。我们可以扩展到C+1个类，因为我们引入了一个背景类。每个类都有一个3x3的得分图，因此一共有(C+1)x3x3个得分图。基于这些得分图我们得到了每个类的得分。之后我们应用一个softmax来计算每个类别上的概率。<br>
下面是数据流。在我们的例子中，令k=3。<br>
![avater](https://cdn-images-1.medium.com/max/1000/1*Gv45peeSM2wRQEdaLG_YoQ.png)

### 回顾
我们从最简单的滑动窗口算法开始。
```python
for window in windows:
    patch = get_patch(image,window)
    results = detector(patch)
```
然后尝试减少窗口的数量以及将工作量移到for循环之外。<br>
```python
ROIs = region_proposal(image)
for ROI in ROIs:
    patch = get_patch(image, ROI)
    results = detector(patch)
```
在第二部分，我们将完全去掉for循环。单次检测器不需要像本文中的分步进行目标检测。<br>

### 拓展阅读，关于FPN, R-FCN以及Mask R-CNN
FPN和R-FCN比我们之前描述的要复杂的多，如果有需要，可以参考：<br>

- [Feature Pyramid Networks(FPN) for object detection](https://medium.com/@jonathan_hui/understanding-feature-pyramid-networks-for-object-detection-fpn-45b227b9106c)
- [Region-based Fully Convolutional Networks (R-FCN).](https://medium.com/@jonathan_hui/understanding-region-based-fully-convolutional-networks-r-fcn-for-object-detection-828316f07c99)
- [Image segmentation with Mask R-CNN](https://medium.com/@jonathan_hui/image-segmentation-with-mask-r-cnn-ebe6d793272)

### 资源
- [Detectron](https://github.com/facebookresearch/Detectron): Facebook Research’s implementation of the Faster R-CNN and Mask R-CNN using Caffe2.
- The official implementation for the [Faster R-CNN](https://github.com/rbgirshick/py-faster-rcnn) in MATLAB.
- [Faster R-CNN](https://github.com/endernewton/tf-faster-rcnn) implementation in TensorFlow.
- [R-FCN](https://github.com/msracver/Deformable-ConvNets) implementation in MXNet.
- [R-FCN](https://github.com/daijifeng001/R-FCN) implementation in Caffe and MATLAB.
- [R-FCN ](https://github.com/xdever/RFCN-tensorflow)implementation in TensorFlow.